ask 1: File Management Script:
Write a Bash script that:
- Creates a directory named "backup" in the user's home directory
- Copies all .txt files from the current directory into the "backup" directory
- Appends the current date and time to the filenames of the copied files.

#Ans:
#!/bin/bash
backup_dir="$HOME/backup"
mkdir -p
"$backup_dir"

timestamp=$(date +"%Y%m%d_%H%M%S")

for file in *.txt;
do
    [ -e "$file" ] || continue

    basename="${file%.txt}"

    cp "$file" "$backup_dir/${basename}_${timestamp}.txt"
done


Ans-2:

#!/bin/bash

LOG_FILE="system_health.log"

# Get CPU usage
cpu_usage=$(top -bn1 | grep "Cpu(s)" | awk '{print 100 - $8}')
cpu_usage=$(printf "%.0f" "$cpu_usage")

# Get memory usage
mem_total=$(free -m | awk '/Mem:/ {print $2}')
mem_available=$(free -m | awk '/Mem:/ {print $7}')
mem_used_percent=$(awk -v t="$mem_total" -v a="$mem_available" \
                   'BEGIN {print 100 - (a*100/t)}')
mem_used_percent=$(printf "%.0f" "$mem_used_percent")

timestamp=$(date +"%Y-%m-%d %H:%M:%S")

status="OK"
message="CPU OK, Memory OK"

# Check if CPU > 80%
if [ "$cpu_usage" -gt 80 ]; then
  status="ALERT"
  message="CPU usage is high"
fi

if [ "$mem_used_percent" -gt 80 ]; then
  status="ALERT"
  if [ "$message" = "CPU usage is high" ]; then
    message="$message and memory is low"
  else
    message="Memory is low"
  fi
fi

echo "$timestamp | CPU: ${cpu_usage}% | RAM used: ${mem_used_percent}% | Status: $status | $message" >> "$LOG_FILE"


In [ ]:
# Ans-3

#!/bin/bash

# Input file with usernames
USER_FILE="user_list.txt"

# Output file for generated credentials
CRED_FILE="credentials.txt"

# Clear old credentials file if it exists
"$CRED_FILE"

# Read each username from the file
while IFS= read -r username;
do
  [ -z "$username" ] && continue

  echo "Creating user: $username"

  # Create the user with a home directory
  useradd -m "$username"

  # Generate a random 12-character password
  password=$(head -c 32 /dev/urandom | base64 | tr -dc 'A-Za-z0-9' | head -c 12)

  # Set the user's password
  echo "${username}:${password}" | chpasswd

  # Save username and password to credentials file
  echo "${username} ${password}" >> "$CRED_FILE"

done
"$USER_FILE"


In [ ]:
# Ans-4

#!/bin/bash
read -p "Enter the directory path to backup: " DIR_PATH

# Check if directory exists
if [ ! -d "$DIR_PATH" ]; then
    echo "Error: Directory does not exist."
    exit 1
fi

# Get current date
DATE=$(date +%F)

# Create backup file name
BACKUP_NAME="backup_$DATE.tar.gz"

# Compress the directory
tar -czf $BACKUP_NAME -C "$(dirname "$DIR_PATH")" "$(basename "$DIR_PATH")"

# Output success message
echo "Backup created successfully: $BACKUP_NAME"


Ans-5:

#!/bin/bash

# File where tasks will be stored
TODO_FILE="todo.txt"

# Ensure the file exists
touch $TODO_FILE

# Function to display menu
show_menu() {
    echo "============================="
    echo "       SIMPLE TO-DO LIST     "
    echo "============================="
    echo "1) View Tasks"
    echo "2) Add Task"
    echo "3) Remove Task"
    echo "4) Exit"
    echo "============================="
}

# Function to view tasks
view_tasks() {
    if [ ! -s $TODO_FILE ]; then
        echo "No tasks found!"
    else
        echo "Your Tasks:"
        nl -w 2 -s ". " $TODO_FILE
    fi
}

# Function to add a task
add_task() {
    read -p "Enter task to add: " task
    echo "$task" >> $TODO_FILE
    echo "Task added!"
}

# Function to remove a task
remove_task() {
    view_tasks
    read -p "Enter task number to remove: " num
    sed -i "${num}d" $TODO_FILE
    echo "Task removed!"
}

# Main program loop
while true; do
    show_menu
    read -p "Choose an option: " choice
    
    case $choice in
        1) view_tasks ;;
        2) add_task ;;
        3) remove_task ;;
        4) echo "Goodbye!"; exit ;;
        *) echo "Invalid option, try again." ;;
    esac

    echo  # Add empty line for readability
done


Ans-6:

#!/bin/bash

# File containing list of packages
PKG_FILE="packages.txt"

# Log file
LOG_FILE="install.log"

# Check if package file exists
if [ ! -f "$PKG_FILE" ]; then
    echo "Error: $PKG_FILE not found!"
    exit 1
fi

# Detect package manager
if command -v apt >/dev/null 2>&1; then
    MANAGER="apt"
    UPDATE_CMD="apt update -y"
    INSTALL_CMD="apt install -y"
elif command -v yum >/dev/null 2>&1; then
    MANAGER="yum"
    UPDATE_CMD="yum update -y"
    INSTALL_CMD="yum install -y"
else
    echo "Unsupported package manager. Only apt and yum supported."
    exit 1
fi

# Update repositories
echo "Updating package repositories..."
eval "$UPDATE_CMD"

echo "Installation started: $(date)" > $LOG_FILE

# Read packages and install them
while IFS= read -r package; do
    if [ -z "$package" ]; then
        continue
    fi

    echo "Installing $package..."
    if eval "$INSTALL_CMD $package" >> $LOG_FILE 2>&1; then
        echo "$package: SUCCESS" | tee -a $LOG_FILE
    else
        echo "$package: FAILED" | tee -a $LOG_FILE
    fi
done < "$PKG_FILE"

echo "Installation completed: $(date)" >> $LOG_FILE
echo "All tasks finished. Check $LOG_FILE for details."


Ans-7:

#!/bin/bash

# Prompt user for a file name
read -p "Enter the text file name: " FILE

# Check if file exists
if [ ! -f "$FILE" ]; then
    echo "Error: File does not exist!"
    exit 1
fi

# Count lines, words, and characters
LINES=$(wc -l < "$FILE")
WORDS=$(wc -w < "$FILE")
CHARS=$(wc -m < "$FILE")

# Find the longest word (ignores punctuation)
LONGEST_WORD=$(tr -d '[:punct:]' < "$FILE" | tr ' ' '\n' | awk '{ if (length > max) { max = length; word = $0 }} END { print word }')

# Display results
echo "================ Text File Statistics ================"
echo "File Name      : $FILE"
echo "Total Lines    : $LINES"
echo "Total Words    : $WORDS"
echo "Total Characters: $CHARS"
echo "Longest Word   : $LONGEST_WORD"
echo "======================================================"